In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [ ]:
%%capture
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

In [ ]:
pd.set_option('max_columns', None)
pd.set_option('display.max_rows', 10)

In [ ]:
path = 'D:\\Desktop\\MLPGD_Capstone_Project\\resources\\aaa_sample_data.xlsx'

df_original = pd.read_excel(path)

In [ ]:
df_original.head(n=2)

In [ ]:
df_original.drop(columns='Column1', inplace=True)

In [ ]:
nullval_ratio = (df_original.isnull().sum() * 100 / len(df_original)).sort_values(ascending=False) # percentage of null values per column


In [ ]:
removed_null_data = df_original.copy()

In [ ]:
threshold = 70.0

In [ ]:
for c in tqdm_notebook(df_original.columns, desc='Process Progress'):

    if nullval_ratio[c] > threshold:

        removed_null_data.drop(columns=c, inplace=True)

In [ ]:
removed_null_data.head(n=2)

In [ ]:
binary_columns = ['FSV CMSI Flag', 'FSV Credit Card Flag', 'FSV Deposit Program Flag', 'FSV Home Equity Flag', 'FSV ID Theft Flag', 'FSV Mortgage Flag', 'INS Client Flag', 'TRV Globalware Flag', 'Responded to Catalog', 'Mail Responder', 'Home Owner', 'Children', 'Gender', 'Bad Address Flag', 'Do Not Direct Mail Solicit', 'Email Available', 'Opt-Out - Publication', 'New Mover Flag', 'Occupant Type', 'Call Canceled', 'Call Killed', 'Cash Call', 'Fleet Indicator', 'Is Duplicate', 'Is NSR',	'Member Match Flag', 'Was Duplicated',	'Was Towed To AAR Referral']

In [ ]:
numericals = pd.DataFrame()

binary = pd.DataFrame()
binary['Household Key'] = removed_null_data['Household Key']

string = pd.DataFrame()
string['Household Key'] = removed_null_data['Household Key']

dates = pd.DataFrame()
dates['Household Key'] = removed_null_data['Household Key']


In [ ]:
for c in tqdm_notebook(removed_null_data.columns, desc='Process Progress'):

    if c in binary_columns:

        binary[c] = removed_null_data[c]
    
    elif removed_null_data.dtypes[c] in ['int64', 'float64']:

        numericals[c] = removed_null_data[c]

    elif removed_null_data.dtypes[c] in ['<M8[ns]']:
        
        dates[c] = removed_null_data[c]

    else:

        string[c] = removed_null_data[c]
    

In [ ]:
numericals = numericals.groupby('Household Key').mean()
binary_numericals = binary.groupby('Household Key').mean() # tem que dropar as colunas binárias numéricas antes de rodar o groupby nas categóricas

In [ ]:
for c in tqdm_notebook(binary.columns, desc='Process Progress'):
    if (c != 'Household Key'):
        if (binary.dtypes[c] in ['int64', 'float64']):

            binary.drop(columns=c, inplace=True)

In [ ]:
for c in tqdm_notebook(dates.columns, desc='Process Progress'):
    if (c != 'Household Key'):
        dates[c] = dates[c].astype('datetime64[ns]')
        dates[c] = dates[c].dt.year
        #dates[c] = dates[c].astype('int64')

In [ ]:
dates  = dates.groupby('Household Key').mean()

In [ ]:
string = string.groupby('Household Key').apply(lambda x: x.mode())

In [ ]:
binary_categoricals = binary.groupby('Household Key').apply(lambda x: x.mode())

In [ ]:
print(len(numericals.index), len(binary_numericals.index), len(binary_categoricals.index), len(dates.index), len(string.index))

In [ ]:
nullval_ratio_numericals = (numericals.isnull().sum(axis=1) * 100 / len(numericals.index)).sort_values(ascending=False) # percentage of null values per row
nullval_ratio_bin_numericals = (binary_numericals.isnull().sum(axis=1) * 100 / len(binary_numericals.index)).sort_values(ascending=False) # percentage of null values per row
nullval_ratio_bin_categoricals = (binary_categoricals.isnull().sum(axis=1) * 100 / len(binary_categoricals.index)).sort_values(ascending=False) # percentage of null values per row
nullval_ratio_dates = (dates.isnull().sum(axis=1) * 100 / len(dates.index)).sort_values(ascending=False) # percentage of null values per row
nullval_ratio_strings = (string.isnull().sum(axis=1) * 100 / len(string.index)).sort_values(ascending=False) # percentage of null values per row

In [ ]:
print('Numericals Null Values','\n',nullval_ratio_numericals, '\n', 'Binary Numericals Null Values','\n', nullval_ratio_bin_numericals, '\n', 'Binary Categorical Null Values','\n', nullval_ratio_bin_categoricals, '\n', 'Dates Null Values','\n', nullval_ratio_dates,'\n', 'String Null Values','\n', nullval_ratio_strings)